### Limma voom analysis of AD/PD dataset, NO SVA

In [1]:
rm(list=ls())
#load necessary libraries 
library(ggplot2)
library(DESeq2)
library("BiocParallel")
parallelFlag=TRUE
register(MulticoreParam(50))
library("IHW")
library("pheatmap")
library(sva)
library(limma)

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colMeans,
    colnames, colSums, dirname, do.call, duplicated, eval, evalq,
    Filter, Find, get, grep, grepl, intersect, is.unsorted, lapply,
    lengths, Map, mapply, match, mget, order, paste, pmax, pmax.int,
    pmin, pmin.int, Position, rank, rbind, Reduce, rowMeans, rownames,
    rowSums, sapply, setdiff, sort, table, tapply, union, unique,
    unsplit, which, which.max, which

## Load data and design

In [2]:
##load ATAC-seq raw read counts
#data=read.table('../adpd.atac.idr.counts.txt.gz',header=TRUE,sep='\t')

##concatenate chrom/start/end columns values to server as rownames for the dataframe of the form chrom_start_end 
#rownames(data)=paste(data$chrom,data$start,data$end,sep="_")
#data$chrom=NULL
#data$start=NULL
#data$end=NULL

#data=data[rowSums(data)>0,]


In [2]:
#load ATAC-seq data normalized counts. These were generated below w/ voom function 
E=read.table("idr.voom.qnorm.txt",header=TRUE,sep='\t',row.names=1)

In [3]:
head(data)

                                                                     
1 function (..., list = character(), package = NULL, lib.loc = NULL, 
2     verbose = getOption("verbose"), envir = .GlobalEnv)            
3 {                                                                  
4     fileExt <- function(x) {                                       
5         db <- grepl("\\\\.[^.]+\\\\.(gz|bz2|xz)$", x)              
6         ans <- sub(".*\\\\.", "", x)                               

In [4]:
#load the metadata
batches=read.table("../batches.filtered.csv",header=TRUE,sep='\t')

In [5]:
Grouping <- factor(paste0(batches$Cohort,".",batches$RegionMod, ".", batches$TypeMod))

In [6]:
batches$Grouping=Grouping

In [7]:
colSums(is.na(batches))

Sample      NewName         Type      TypeMod       Region    RegionMod 
           0            0            0            0            0            0 
      Cohort TissueCenter        Batch       Gender  expired_age          PMI 
           0            0            0            0            0            4 
        ApoE     Grouping 
          17            0

In [8]:
#SVA can't handle NA values, so we have no choice but to interpolate to the mode for missing entries in PMI & ApoE 
batches$ApoE[is.na(batches$ApoE)]='3_3'
batches$PMI[is.na(batches$PMI)]=mean(na.omit(batches$PMI))

In [9]:
#Now, we include all covariates and surrogate variables 
mod=model.matrix(~Grouping+Gender+expired_age+PMI+TissueCenter+ApoE+Batch,data=batches)

### fit model with limma 

In [10]:
fit <- lmFit(E,mod)

Coefficients not estimable: TissueCenterWUSTL BatchPD_X031 


Warning message:
“Partial NA coefficients for 385725 probe(s)”

In [13]:
colnames(mod)


[1] "(Intercept)"          "GroupingAD.CAUD.CTRL" "GroupingAD.CAUD.LOAD"
 [4] "GroupingAD.HIPP.ADAD" "GroupingAD.HIPP.CTRL" "GroupingAD.HIPP.LOAD"
 [7] "GroupingAD.PARL.ADAD" "GroupingAD.PARL.CTRL" "GroupingAD.PARL.LOAD"
[10] "GroupingAD.SMTG.ADAD" "GroupingAD.SMTG.CTRL" "GroupingAD.SMTG.LOAD"
[13] "GroupingPD.CAUD.ADPD" "GroupingPD.CAUD.CTRL" "GroupingPD.CAUD.LOPD"
[16] "GroupingPD.HIPP.ADPD" "GroupingPD.HIPP.CTRL" "GroupingPD.HIPP.LOPD"
[19] "GroupingPD.MDFG.ADPD" "GroupingPD.MDFG.CTRL" "GroupingPD.MDFG.LOPD"
[22] "GroupingPD.PTMN.ADPD" "GroupingPD.PTMN.CTRL" "GroupingPD.PTMN.LOPD"
[25] "GroupingPD.SMTG.ADPD" "GroupingPD.SMTG.CTRL" "GroupingPD.SMTG.LOPD"
[28] "GroupingPD.SUNI.ADPD" "GroupingPD.SUNI.CTRL" "GroupingPD.SUNI.LOPD"
[31] "GenderMale"           "expired_age"          "PMI"                 
[34] "TissueCenterUW"       "TissueCenterWUSTL"    "ApoE2_3"             
[37] "ApoE3_3"              "ApoE3_4"              "ApoE4_4"             
[40] "BatchAD_X002"         "BatchAD_X003"         "BatchAD_X004"        
[43] "BatchAD_X005"         "BatchAD_X006"         "BatchAD_X007"        
[46] "BatchAD_X008"         "BatchAD_X009"         "BatchAD_X010"        
[49] "BatchAD_X011"         "BatchAD_X012"         "BatchAD_X013"        
[52] "BatchAD_X014"         "BatchAD_X015"         "BatchAD_X016"        
[55] "BatchAD_X017"         "BatchAD_X018"         "BatchPD_X001"        
[58] "BatchPD_X002"         "BatchPD_X003"         "BatchPD_X004"        
[61] "BatchPD_X005"         "BatchPD_X006"         "BatchPD_X007"        
[64] "BatchPD_X008"         "BatchPD_X009"         "BatchPD_X010"        
[67] "BatchPD_X011"         "BatchPD_X012"         "BatchPD_X013"        
[70] "BatchPD_X014"         "BatchPD_X015"         "BatchPD_X016"        
[73] "BatchPD_X017"         "BatchPD_X018"         "BatchPD_X019"        
[76] "BatchPD_X020"         "BatchPD_X021"         "BatchPD_X022"        
[79] "BatchPD_X023"         "BatchPD_X024"         "BatchPD_X025"        
[82] "BatchPD_X026"         "BatchPD_X027"         "BatchPD_X028"        
[85] "BatchPD_X029"         "BatchPD_X030"         "BatchPD_X031"

In [14]:
colnames(fit$coefficients)

[1] "(Intercept)"          "GroupingAD.CAUD.CTRL" "GroupingAD.CAUD.LOAD"
 [4] "GroupingAD.HIPP.ADAD" "GroupingAD.HIPP.CTRL" "GroupingAD.HIPP.LOAD"
 [7] "GroupingAD.PARL.ADAD" "GroupingAD.PARL.CTRL" "GroupingAD.PARL.LOAD"
[10] "GroupingAD.SMTG.ADAD" "GroupingAD.SMTG.CTRL" "GroupingAD.SMTG.LOAD"
[13] "GroupingPD.CAUD.ADPD" "GroupingPD.CAUD.CTRL" "GroupingPD.CAUD.LOPD"
[16] "GroupingPD.HIPP.ADPD" "GroupingPD.HIPP.CTRL" "GroupingPD.HIPP.LOPD"
[19] "GroupingPD.MDFG.ADPD" "GroupingPD.MDFG.CTRL" "GroupingPD.MDFG.LOPD"
[22] "GroupingPD.PTMN.ADPD" "GroupingPD.PTMN.CTRL" "GroupingPD.PTMN.LOPD"
[25] "GroupingPD.SMTG.ADPD" "GroupingPD.SMTG.CTRL" "GroupingPD.SMTG.LOPD"
[28] "GroupingPD.SUNI.ADPD" "GroupingPD.SUNI.CTRL" "GroupingPD.SUNI.LOPD"
[31] "GenderMale"           "expired_age"          "PMI"                 
[34] "TissueCenterUW"       "TissueCenterWUSTL"    "ApoE2_3"             
[37] "ApoE3_3"              "ApoE3_4"              "ApoE4_4"             
[40] "BatchAD_X002"         "BatchAD_X003"         "BatchAD_X004"        
[43] "BatchAD_X005"         "BatchAD_X006"         "BatchAD_X007"        
[46] "BatchAD_X008"         "BatchAD_X009"         "BatchAD_X010"        
[49] "BatchAD_X011"         "BatchAD_X012"         "BatchAD_X013"        
[52] "BatchAD_X014"         "BatchAD_X015"         "BatchAD_X016"        
[55] "BatchAD_X017"         "BatchAD_X018"         "BatchPD_X001"        
[58] "BatchPD_X002"         "BatchPD_X003"         "BatchPD_X004"        
[61] "BatchPD_X005"         "BatchPD_X006"         "BatchPD_X007"        
[64] "BatchPD_X008"         "BatchPD_X009"         "BatchPD_X010"        
[67] "BatchPD_X011"         "BatchPD_X012"         "BatchPD_X013"        
[70] "BatchPD_X014"         "BatchPD_X015"         "BatchPD_X016"        
[73] "BatchPD_X017"         "BatchPD_X018"         "BatchPD_X019"        
[76] "BatchPD_X020"         "BatchPD_X021"         "BatchPD_X022"        
[79] "BatchPD_X023"         "BatchPD_X024"         "BatchPD_X025"        
[82] "BatchPD_X026"         "BatchPD_X027"         "BatchPD_X028"        
[85] "BatchPD_X029"         "BatchPD_X030"         "BatchPD_X031"

###  PD, CAUD

In [16]:
#create contrasts of interest 
pd.caud.cont.matrix=makeContrasts(
                          pd_caud_adpd_vs_lopd="GroupingPD.CAUD.ADPD - GroupingPD.CAUD.LOPD",
                          pd_caud_adpd_vs_ctrl="GroupingPD.CAUD.ADPD - GroupingPD.CAUD.CTRL",
                          pd_caud_lopd_vs_ctrl="GroupingPD.CAUD.LOPD - GroupingPD.CAUD.CTRL",
                          levels=mod)


Warning message in makeContrasts(pd_caud_adpd_vs_lopd = "GroupingPD.CAUD.ADPD - GroupingPD.CAUD.LOPD", :
“Renaming (Intercept) to Intercept”

In [17]:
fit2=contrasts.fit(fit,pd.caud.cont.matrix)
e=eBayes(fit2)
comparisons=colnames(pd.caud.cont.matrix)

Warning message in contrasts.fit(fit, pd.caud.cont.matrix):
“row names of contrasts don't match col names of coefficients”

In [18]:
comparisons

[1] "pd_caud_adpd_vs_lopd" "pd_caud_adpd_vs_ctrl" "pd_caud_lopd_vs_ctrl"

In [25]:
for(i in seq(1,length(comparisons)))
{
  tab<-topTable(e, number=nrow(e),coef=i,p.value = 0.05)
  up=sum(tab$logFC>0)
  down=sum(tab$logFC<0)
  sig=nrow(tab)
  curtitle=paste(comparisons[i],'\n','sig:',sig,'\n','up:',up,'\n','down:',down,'\n')
  print(curtitle)
  vals=topTable(e,number=nrow(e),coef=i)
  vals$pscaled=-10*log10(vals$adj.P.Val)
  vals$sig=vals$adj.P.Val<0.05
  png(paste("volcano_diff",comparisons[i],".png",sep=""))
  print(ggplot(data=vals,
               aes(y=vals$pscaled,x=vals$logFC,color=vals$sig))+
   geom_point()+
   xlab("log2(FC)")+
   ylab("-10*log10(pval)")+
   ggtitle(curtitle))
  dev.off() 
  write.table(tab,file=paste("diff_",comparisons[i],".tsv",sep=""),quote=FALSE,sep='\t',row.names = TRUE,col.names = TRUE)
}
    

[1] "pd_caud_adpd_vs_lopd \n sig: 0 \n up: 0 \n down: 0 \n"
[1] "pd_caud_adpd_vs_ctrl \n sig: 659 \n up: 414 \n down: 245 \n"
[1] "pd_caud_lopd_vs_ctrl \n sig: 3 \n up: 0 \n down: 3 \n"


In [ ]:
### PD, 